In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.appName("spark").getOrCreate()

# create the Spark Context
sc = spark.sparkContext

/home/ljj0512/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


## count total number of genres

In [1]:
import sqlite3
con = sqlite3.connect("./Books.db")
cur = con.cursor()
cur.execute("SELECT genres FROM Foreign_book")
data = cur.fetchall()
print(len(data))
print(type(data))
print(type(data[0]))
print(data[0])

52478
<class 'list'>
<class 'tuple'>
('Young Adult, Fiction, Dystopia, Fantasy, Science Fiction, Romance, Adventure, Teen, Post Apocalyptic, Action',)


In [3]:
rdd = sc.parallelize(data).map(lambda genres: genres[0])
print(rdd.take(1))
print(type(rdd.take(1)))
print(type(rdd.take(1)[0]))

['Young Adult, Fiction, Dystopia, Fantasy, Science Fiction, Romance, Adventure, Teen, Post Apocalyptic, Action']
<class 'list'>
<class 'str'>


In [4]:
rdd1 = rdd.map(lambda line: tuple(line.split(", ")))
print(rdd1.take(1))
print(rdd1.count())

[('Young Adult', 'Fiction', 'Dystopia', 'Fantasy', 'Science Fiction', 'Romance', 'Adventure', 'Teen', 'Post Apocalyptic', 'Action')]
52478


In [5]:
rdd2 = rdd1.flatMap(lambda x: x)
print(rdd2.take(5))
print(rdd2.count())

['Young Adult', 'Fiction', 'Dystopia', 'Fantasy', 'Science Fiction']
412341


In [17]:
genre_set = set({})

for i, value in enumerate(rdd2.collect()):
    genre_set.add(value)

print("total genres:",len(genre_set))

total genres: 983


In [16]:
genre_dic = {}

for i, value in enumerate(rdd2.collect()):
    if value in genre_dic:
        genre_dic[value] += 1
    else:
        genre_dic[value] = 1

print("total genres:",len(genre_dic))
it = zip(genre_dic.keys(), genre_dic.values())
print(next(it))
print(next(it))
print(next(it))
print(next(it))
print(next(it))

total genres: 983
('Young Adult', 11869)
('Fiction', 31638)
('Dystopia', 1692)
('Fantasy', 15046)
('Science Fiction', 5374)


## contruct matrix for content-based recommendation

In [18]:
#
#

In [22]:
genres_list = (data["genres"].iloc[:].tolist())

In [20]:
import sqlite3
conn = sqlite3.connect("Books.db")
cur = conn.cursor()
cur.execute("SELECT title FROM Korean_book WHERE rowid=24416")
data = cur.fetchall()
print(data)
conn.commit()
conn.close()

[]


In [23]:
import numpy as np
x = np.array([10,0,1,1])
y = np.array([9,1,0,0])
z = np.array([3,0,1,1])
from scipy.spatial import distance
print(distance.cosine(x, y))
print(distance.cosine(x, x))
print(distance.cosine(x, z))
print(distance.cosine(y, z))

0.01590846210843544
0.0
0.04466970555754857
0.10099833641376377


In [28]:
import math
def consine_similarity(x, y):
    return x.dot(y)/(np.sqrt((x**2).sum())*np.sqrt((y**2).sum()))

print(consine_similarity(x,y))
print(consine_similarity(x,z))
# print(consine_similarity(y,z))


0.9840915378915646
0.9553302944424514


In [13]:
import pandas as pd

data = pd.read_csv("./data/foreign-books.csv")
title = data["title"].iloc[0]

print(len(data))
print(data.iloc[0])
print(data["coverImg"].iloc[0])
print(data["description"].iloc[0])

52478
bookId                                       2767052-the-hunger-games
title                                                The Hunger Games
series                                            The Hunger Games #1
author                                                Suzanne Collins
rating                                                           4.33
description         WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...
language                                                      English
isbn                                                    9780439023481
genres              ['Young Adult', 'Fiction', 'Dystopia', 'Fantas...
characters          ['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...
bookFormat                                                  Hardcover
edition                                                 First Edition
pages                                                             374
publisher                                            Scholastic Press
publishDate   

In [6]:
import pandas as pd
data = pd.read_csv("./data/foreign-books.csv")
title = data["title"].iloc[0]
genres = data["genres"].iloc[0].replace("[","").replace("]","").replace("\'","")
author = data["author"].iloc[0]
rating = data["rating"].iloc[0]
publisher = data["publisher"].iloc[0]
p_date = data["publishDate"].iloc[0]
pages = data["pages"].iloc[0]
language = data["language"].iloc[0]
description = data["description"][0]
imgUrl = data["coverImg"][0]

print(title)
print(genres)
print(author)
print(rating)
print(publisher)
print(p_date)
print(pages)
print(language)
print(description)
print(imgUrl)
print(len(data))

The Hunger Games
Young Adult, Fiction, Dystopia, Fantasy, Science Fiction, Romance, Adventure, Teen, Post Apocalyptic, Action
Suzanne Collins
4.33
Scholastic Press
09/14/08
374
English
WINNING MEANS FAME AND FORTUNE.LOSING MEANS CERTAIN DEATH.THE HUNGER GAMES HAVE BEGUN. . . .In the ruins of a place once known as North America lies the nation of Panem, a shining Capitol surrounded by twelve outlying districts. The Capitol is harsh and cruel and keeps the districts in line by forcing them all to send one boy and once girl between the ages of twelve and eighteen to participate in the annual Hunger Games, a fight to the death on live TV.Sixteen-year-old Katniss Everdeen regards it as a death sentence when she steps forward to take her sister's place in the Games. But Katniss has been close to dead before—and survival, for her, is second nature. Without really meaning to, she becomes a contender. But if she is to win, she will have to start making choices that weight survival against human